# Sudoku validation model
In this notebook, a model is created to validate a Sudoku. Tensoflow is used to create a CNN designed for the mentioned purpose.

## Imports

In [2]:
import copy
import os
import datetime
import _pickle as cPickle
from IPython.display import clear_output
import random
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
import pySudoku

MODELS_PATH = './models/validation'
DATASET_PATH = './dataset/validation'

## Loading the dataset

Following code is used to load a labeled valid/invalid sudoku dataset, as well as create features and labels variables for train/test/validation.

In [3]:
train_size = 0.8
test_size = 0.5

sudoku_dataset = []
if os.path.exists(DATASET_PATH):
    for filename in os.listdir(DATASET_PATH):
        if(filename.endswith('.cPickle')):
            input_path = os.path.join(DATASET_PATH, filename)
            for sudoku in cPickle.load(open(input_path, 'rb')):
                sudoku_dataset.append(sudoku)
random.shuffle(sudoku_dataset)

# Prepare features and labels arrays
x = np.array([np.array(x[0]) for x in sudoku_dataset])/9
y = np.array([(1 if x[1] == 1 else 0, 1 if x[1] == 0 else 0)
             for x in sudoku_dataset])

x_train, x_rem, y_train, y_rem = train_test_split(x, y, train_size=train_size)

x_valid, x_test, y_valid, y_test = train_test_split(
    x_rem, y_rem, test_size=test_size)


## Load existing model

In [4]:
MODEL_FILENAME = 'sudoku-validation-v1.0.h5'

model_path = os.path.join(MODELS_PATH, MODEL_FILENAME)
sudoku_model = tf.keras.models.load_model(model_path)


Metal device set to: Apple M1


2021-08-19 15:14:04.545097: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-08-19 15:14:04.545183: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


## Create the model

The model is a CNN that applies filter to a Sudoku to determine whether it is valid or not. After defining the model architecture, it is trained using the dataset loaded on previous steps.

In [5]:
sudoku_model = tf.keras.models.Sequential()
sudoku_model.add(tf.keras.layers.InputLayer(input_shape=(9,9)))
sudoku_model.add(tf.keras.layers.Reshape(target_shape=(1,9,9)))

sudoku_model.add(tf.keras.layers.Conv2D(filters=9, kernel_size=9, padding='same', activation='relu', input_shape=(1,9,9)))
sudoku_model.add(tf.keras.layers.BatchNormalization())
sudoku_model.add(tf.keras.layers.LeakyReLU())

sudoku_model.add(tf.keras.layers.Conv2D(filters=9, kernel_size=9, padding='same', activation='relu', input_shape=(1,9,9)))
sudoku_model.add(tf.keras.layers.BatchNormalization())
sudoku_model.add(tf.keras.layers.LeakyReLU())

sudoku_model.add(tf.keras.layers.Flatten())
sudoku_model.add(tf.keras.layers.Dropout(0.5))
sudoku_model.add(tf.keras.layers.Dense(2, activation='softmax'))
 
sudoku_model.compile(optimizer='sgd',
                     loss=tf.keras.losses.MeanSquaredError(),
                     metrics=['accuracy'])

sudoku_model.fit(x, y, batch_size=128, epochs=5, validation_data=(x_valid, y_valid))


2021-08-19 15:14:04.827358: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-08-19 15:14:04.827521: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2021-08-19 15:14:04.945074: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Epoch 1/6
1844/1844 [==============================] - ETA: 0s - loss: 0.2157 - accuracy: 0.6394

2021-08-19 15:14:32.639531: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


1844/1844 [==============================] - 29s 16ms/step - loss: 0.2157 - accuracy: 0.6394 - val_loss: 0.5048 - val_accuracy: 0.4878
Epoch 2/6
1844/1844 [==============================] - 29s 16ms/step - loss: 0.0910 - accuracy: 0.8897 - val_loss: 0.2903 - val_accuracy: 0.4969
Epoch 3/6
1844/1844 [==============================] - 29s 16ms/step - loss: 0.0403 - accuracy: 0.9557 - val_loss: 0.0379 - val_accuracy: 0.9791
Epoch 4/6
1844/1844 [==============================] - 29s 16ms/step - loss: 0.0271 - accuracy: 0.9704 - val_loss: 0.0604 - val_accuracy: 0.9849
Epoch 5/6
1844/1844 [==============================] - 29s 16ms/step - loss: 0.0240 - accuracy: 0.9741 - val_loss: 0.0188 - val_accuracy: 0.9832
Epoch 6/6
1844/1844 [==============================] - 29s 16ms/step - loss: 0.0180 - accuracy: 0.9808 - val_loss: 0.0102 - val_accuracy: 0.9903


## Test the model
Testing the model using the test dataset.

In [6]:
result = sudoku_model.evaluate(x=x_test, y=y_test)

738/738 [==============================] - 4s 6ms/step - loss: 0.0116 - accuracy: 0.9881


## Use the model to predict a single Sudoku
The following lines of code create a new Sudoku, using `PySudoku` library, then predicts whether it's valid or not using the model. As a next step, the valid sudoku is modified to make a repeated value on the first row of the Sudoku (it makes it invalid), then predict again with the model.

In [7]:
sudoku_tools = pySudoku.PySudoku()

brand_new_sudoku = sudoku_tools.get_empty_sudoku()
brand_new_sudoku_solved = sudoku_tools.solve_sudoku(brand_new_sudoku)['sudoku']
#sudoku_tools.print_sudoku(brand_new_sudoku_solved)
 
# Validate that the brand new Sudoku does not exist in the training dataset
print("Brand new sudoku exists in dataset:",
      np.array(brand_new_sudoku_solved) in x)
# Let normalize the sudoku values dividing by 10 each cell.
brand_new_sudoku_solved_normalized = (np.array(brand_new_sudoku_solved)/9).tolist()
 
# Let's try to predict if it's correct...
print('Using model to predict if sudoky is valid or not...')
prediction = sudoku_model.predict([brand_new_sudoku_solved_normalized])[0]
print('Prediction raw:', prediction) 
print('Prediction result: ', 
      'Valid' if prediction[0]>prediction[1] else 'Invalid',
      'sudoku')
 
print('Making brand new sudoku invalid by duplicating a value...')
brand_new_sudoku_solved[0][0]=brand_new_sudoku_solved[0][1]
#sudoku_tools.print_sudoku(brand_new_sudoku_solved)
 
# Let normalize the sudoku values again.
brand_new_sudoku_solved_normalized = (np.array(brand_new_sudoku_solved)).tolist()
 
# Let's try to predict if it's incorrect...
print('Using model to predict if sudoky is valid or not...')
prediction = sudoku_model.predict([brand_new_sudoku_solved_normalized])[0]
print('Prediction raw:', prediction)
print('Prediction result: ', 
      'Valid' if prediction[0]>prediction[1] else 'Invalid',
      'sudoku')

Brand new sudoku exists in dataset: True
Using model to predict if sudoky is valid or not...
Prediction raw: [0.9178636  0.08213634]
Prediction result:  Valid sudoku
Making brand new sudoku invalid by duplicating a value...
Using model to predict if sudoky is valid or not...
Prediction raw: [0. 1.]
Prediction result:  Invalid sudoku


2021-08-19 15:17:04.380655: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-08-19 15:17:04.428127: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
